### 12.1 빅데이터 배치 프로그램 만들기

#### 실습 프로젝트 구성
1. nodejs의 fs패키지가 제공하는 비동기 방식의 API들의 Promise 방식 구현
1. range, zip같은 유틸리티 함수 구현
1. chance패키지를 사용한 dummy 데이터 생성 코드 구현
1. CSV파일 포맷 데이터 읽고 쓰는 코드 구현

```bash
npm init --y
npm i -D typescript ts-node @types/node

// 디렉토리생성/삭제 기능 지원
npm i -S mkdirp rimraf
npm i -D @types/mkdirp @types/rimraf

npm i -S chance
npm i -D @types/chance

tsc --init

mkdir -p src/1201/fileApi
mkdir src/1201/fake
mkdir src/1201/csv
mkdir src/1201/utils
mkdir src/1201/test
```

#### csv 파일과 생성기
* csv 파일 포맷 예

```csv
name,email,ip,phone
Leona Alexander,ikero@gmail.com,192.168.1.100,010-9999-8888
```

#### nodejs에서 프로그램 명령 줄 인수 읽기
* nodejs에는 process 내장 객체 제공

##### src/1201/test/processArgs-test.ts 

```ts
process.argv.forEach((val: string, index: number) => {
  console.log(index + ': ' + val);
})
```

##### src/1201/utils/getFileNameAndNumber.ts 

```ts
export type FileNameAndNumber = [string, number]

export const getFileNameAndNumber = (defaultFileName: string, 
  defaultNumberOffFakeData: number): FileNameAndNumber => {
    const [bin, node, filename, numberOfFakeData] = process.argv
    return [filename || defaultFileName, 
      numberOfFakeData ? parseInt(numberOfFakeData, 10) : defaultNumberOffFakeData]
  }
```

##### src/1201/test/getFileNameAndNumber-test.ts 

```ts
import { getFileNameAndNumber } from "../utils/getFileNameAndNumber";

const [filename, numberOfFakeItems] = getFileNameAndNumber('./data/fake.csv', 100000);
console.log(filename, numberOfFakeItems)
```

##### fake data  생성하기
```bash
ts-node src/1201/test/processArgs-test.ts data/fake.csv 100000
```

#### 파일처리 비동기 함수를 프로미스로 구현하기

##### 1) fs.access API로 디렉토리나 파일 확인하기
##### src/1201/fileApi/fileExists.ts

```ts
import * as fs from 'fs'

export const fileExists = (filepath: string): Promise<boolean> => 
  new Promise(resolve => fs.access(filepath, error => resolve(error ? false : true)))
```

##### src/1201/test/fileExists-test.ts

```ts
import { fileExists } from "../fileApi/fileExists";

const exists = async(filepath) => {
  const result = await fileExists(filepath)
  console.log(`${filepath} ${result ? 'exists' : 'not exists'}`)
}

exists('./package.json')
exists('./package')
```
```bash
ts-node src/1201/test/fileExists-test.ts
```

##### 2) mkdirp 패키지로 디렉터리 생성함수 만들기
* mkdirp는 여러 경로의 디렉토리를 한번에 생성하도록 지원
* `mkdir -p`처럼 동작하는 API를 제공

##### src/1201/fileApi/mkdir.ts

```ts
import { mkdirp } from 'mkdirp'
import { fileExists } from './fileExists'

export const mkdir = (dirname: string): Promise<string> =>
  new Promise(async (resolve, reject) => {
    try {
      const alreadyExists = await fileExists(dirname);
      if (alreadyExists) {
        resolve(dirname);
      } else {
        const createdDir = await mkdirp(dirname); // mkdirp가 반환하는 값을 저장
        resolve(createdDir || dirname); // `mkdirp`가 void일 경우 대비하여 dirname을 반환
      }
    } catch (error) {
      reject(error);
    }
  });
```


##### src/1201/test/mkdir-test.ts

```ts
import {mkdir} from '../fileApi/mkdir'

const makeDataDir = async(dirname: string) => {
  let result = await mkdir(dirname)
  console.log(`'${result}' dir created`) // './data/today' dir created
}
makeDataDir('./data/today')
```

##### 3) rimfaf 패키지로 디렉터리 삭제 함수 만들기
* mkdirp는 여러 경로의 디렉토리를 한번에 생성하도록 지원
* `mkdir -p`처럼 동작하는 API를 제공

##### src/1201/fileApi/rmdir.ts

```ts
import { rimraf } from "rimraf";
import { fileExists } from "./fileExists";

export const rmdir = (dirname: string): Promise<string> =>
  new Promise(async(resolve, reject) => {
    try {
      const alreadyExists = await fileExists(dirname)
      if(!alreadyExists) {
        resolve(dirname)
      } else {
        await rimraf(dirname);
        resolve(dirname);
      }
    } catch (error) {
      reject(error);
    }
  })
```

##### src/1201/fileApi/rmdir-test.ts


```ts
import { rmdir } from '../fileApi/rmdir'

const deleteDataDIr = async (dir) => {
  const result = await rmdir(dir)
  console.log(`${result} dir deleted~~`)
}

deleteDataDIr('./data/today')
```

##### 4) fs.writeFile API로 파일 생성하기

##### src/1201/fileApi/writeFile.ts

```ts
import * as fs from 'fs'

export const writeFile = (filename: string, data: any): Promise<any> =>
  new Promise((resolve, reject) => {
    fs.writeFile(filename, data, 'utf8', (error: Error) => {
      error ? reject(error) : resolve(data);
    })
  })
```

##### src/1201/test/writeFile-test.ts

```ts
import * as path from 'path'
import { writeFile } from '../fileApi/writeFile'
import { mkdir } from '../fileApi/mkdir'
import mkdirp = require('mkdirp')

const writeTest = async(filename: string, data: any) => {
  const result = await writeFile(filename, data);
  console.log(`write ${result} to ${filename}`)
}

// ✅ JSON.stringify(value, replacer, space) 파라미터 설명
// value	 : JSON 문자열로 변환할 객체
// replacer: 특정 속성을 필터링할 함수 또는 배열 (기본값: null, 즉 모든 속성 포함)
// space	 : JSON을 보기 쉽게 들여쓰기할 공백 또는 개행 문자 (기본값: 0, 즉 한 줄 출력)
mkdir('./data')
  .then(s => writeTest('./data/hello.txt', 'Hello World!!'))
  .then(s => writeTest('./data/test.json', JSON.stringify({name: '홍길동', age: 1000}, null, 2)))
  .catch((e: Error) => console.log(e.message))
```

##### 5) fs.readFile API로 파일 읽기

##### src/1201/fileApi/readFile.ts

```ts
import * as fs from 'fs'

export const readFile = (filename: string): Promise<string> =>
  new Promise<any>((resolve, reject) => {
    fs.readFile(filename, 'utf8', (error: Error, data: any) => {
      error ? reject(error) : resolve(data)
    })
  })
```

##### src/1201/test/readFile-test.ts

```ts
import { readFile } from "../fileApi/readFile";

const readTest = async(filename: string) => {
  const result = await readFile(filename)
  console.log(`read ${result} from ${filename} file.`)
}

readTest('/data/hello.txt')
  .then(s => readTest(('/data/test.json'))
  .catch((e: Error) => console.log(e.message)))
```
##### 6) fs.appendFile API로 파일 내용 추가하기
##### src/1201/fileApi/appendFile.ts

```ts
import * as fs from 'fs'

export const appendFile = (filename: string, data: any) : Promise<any> =>
  new Promise((resolve, reject) => {
    fs.appendFile(filename, data, 'utf8', (error: Error) => {
      error ? reject(error) : resolve(data)
    })
  })
```

##### src/1201/test/appendFile-test.ts

```ts
import * as fs from 'fs'
import { appendFile } from '../fileApi/appendFile'
import { mkdir } from '../fileApi/mkdir'

const appendTest = async(filename: string, data: any) => {
  const result = await appendFile(filename, data)
  console.log(`append ${result} to ${filename}`)
}

mkdir('./data')
  .then(s => appendTest('./data/hello.txt', 'Hi 홍길동???')
  .catch((e: Error) => console.log(e.message)))
```
##### 7) fs.unlink API로 파일 삭제 하기
##### src/1201/fileApi/deleteFile.ts

```ts
import * as fs from 'fs'
import { fileExists} from './fileExists'

export const deleteFile = (filename: string) : Promise<string> => 
  new Promise<any>(async(resolve, reject) => {
    const alreadyExists = await fileExists(filename)
    !alreadyExists ? resolve(filename) :
      fs.unlink(filename, (error) => error ? reject(error) : resolve(filename))
  })
```

##### src/1201/test/deleteFile-test.ts

```ts
import { deleteFile } from '../fileApi/deleteFile'
import { rmdir } from '../fileApi/rmdir'

const deleteTest = async(filename: string) => { 
  const result = await deleteFile(filename)
  console.log(`delete ${result} file!`)
}

Promise.all([deleteTest('./data/hello.txt'), deleteTest('./data/test.json')])
  .then(s => rmdir('./data'))
  .then(dirname => console.log(`delete ${dirname} dir!`))
  .catch((e: Error) => console.log(e.message))
```

##### 8) fsrc/1201/fileApi/index.ts 만들기
##### src/1201/fileApi/index.ts

```ts
import { fileExists } from './fileExists'
import { mkdir } from './mkdir'
import { rmdir } from './rmdir'
import { writeFile } from './writeFile'
import { readFile } from './readFile'
import { appendFile } from './appendFile'
import { deleteFile } from './deleteFile'

export {fileExists, mkdir, rmdir, writeFile, readFile, appendFile, deleteFile}
```

#### dummy data 만들기
##### src/1201/fake/IFake.ts

```ts
export interface IFake {
  name: string
  email: string
  sentence: string
  profession: string
  birthday: Date
}
```
##### src/1201/fake/makeFakeData.ts

```ts
import Chance from 'chance' 
import { IFake } from './IFake'

const c = new Chance

export const makeFakeData = () : IFake => ({
  name: c.name(),
  email: c.email(),
  profession : c.profession(),
  birthday: c.birthday(),
  sentence: c.sentence()
})

export { IFake }
```
##### src/1201/test/makeFakeData-test.ts

```ts
import { makeFakeData, IFake } from '../fake/makeFakeData'

const fakeData: IFake = makeFakeData()
console.log(fakeData);
```


#### Objectkeys와 Object.vlues 함수 사용하기

##### src/1201/test/keys-values-test.ts

```ts
import { IFake, makeFakeData } from '../fake/makeFakeData'

const data: IFake = makeFakeData()
const keys = Object.keys(data)
const values = Object.values(data)

console.log('keys:', keys, '\nvalues: ',  values)
```

#### CSV파일 만들기

##### src/1201/utils/range.ts

```ts
export function* range(max: number, min: number = 0) {
  while(min < max)
    yield min++
}
``
##### src/1201/fake/writeCsvFormatFakeData.ts

```ts
import * as path from 'path'
import { IFake, makeFakeData } from './makeFakeData'
import { mkdir, writeFile, appendFile } from '../fileApi'
import { range } from '../utils/range'

export const writeCsvFormatFakeData = async(filename: string, 
  numberOfItems: number): Promise<string> => {
    const dirname = path.dirname(filename)
    await mkdir(dirname)
    const comma = ',', newLine= '\n'
    for(let n of range(numberOfItems)) {
      const fake: IFake = makeFakeData()
      if(n === 0) {
        const keys = Object.keys(fake).join(comma)
        await writeFile(filename, keys)
      }
      const values = Object.values(fake).join(comma)
      await appendFile(filename, newLine + values)
    }
    return `write ${numberOfItems} items to ${filename} file`
  }
```
##### src/1201/fake/index.ts

```ts
import { IFake, makeFakeData } from './makeFakeData'
import { writeCsvFormatFakeData } from './writeCsvFormatFakeData'

export { IFake, makeFakeData, writeCsvFormatFakeData }
```

#### 데이터를 CSV 파일에 쓰기

##### src/1201/writeCsv.ts

```ts
import {writeCsvFormatFakeData} from './fake'
import {getFileNameAndNumber} from './utils/getFileNameAndNumber'

const [filename, numberOfFakeData] = getFileNameAndNumber('./data/fake', 1)
const csvFilename = `${filename}-${numberOfFakeData}.csv`
writeCsvFormatFakeData(csvFilename, numberOfFakeData)
  .then(result => console.log(result))
  .catch((e: Error) => console.log(e.message))
```
```bash
ts-node src/1201/writeCsv.ts 
ts-node src/1201/writeCsv.ts data/fake 100000
```

#### zip 함수 만들기

##### src/1201/utils/zip.ts

```ts
export const zip = (keys: string[], values: any) => {
  const makeObject = (key: string, value:any) => ({[key]: value})
  const mergeObject = (a: any[]) => a.reduce((sum, val) => ({...sum, ...val}), {})

  let tmp = keys
    .map((key, index) => [key, values[index]])
    .filter(a => a[0] && a[1])
    .map(a => makeObject(a[0], a[1]))
  
    return mergeObject(tmp)
}
```

##### src/1201/utils/index.ts

```ts
import { getFileNameAndNumber, FileNameAndNumber } from './getFileNameAndNumber'
import { range } from './range'
import { zip } from './zip'

export { getFileNameAndNumber, FileNameAndNumber, range, zip }
```


##### src/1201/test/zip-test.ts

```ts
import {zip} from '../utils'
import {makeFakeData, IFake} from '../fake'
const data = makeFakeData()
const keys = Object.keys(data), values = Object.values(data)

const fake: IFake = zip(keys, values) as IFake
console.log(fake)
```

#### 생성기 코드를 구현할 떄 주의할 점
* 생성기를 작성할 때는 `yield식은 생성기 본문에서만 사용할 수 있다.` 오류에 주의할 것
* 생성기를 구현할 때는 fs.readFile과 같은 비동기함수를 사용할 수 없다.

#### 데이터를 CSV 파일에 읽기

##### src/1201/fileApi/readFileGenerator.ts
* 파일읽기를 생성기방식으로 구현할 때 fs.readFile을 사용하지 못하므로 fs.readFile방식으로 구현할 경우
* 시스템메모리를 많이 사용하는 문제가 발생하므로 대량의 데이터를 읽을 경우는 바람직하지 못하다
* 결론적으로 파일을 한 줄씩 읽는 방식으로 생성기를 구현하는 것이다.

```ts
import * as fs from 'fs'

export function *readFileGenerator(filename: string): any {
  let fd:any

  try {
    fd = fs.openSync(filename, "rs")
    const stats = fs.fstatSync(fd)
    const bufferSize = Math.min(stats.size, 1024)
    const buffer = Buffer.alloc(bufferSize+4)
    let filepos = 0, line

    while(filepos>-1) {
      [line, filepos] = readLine(fd, buffer, bufferSize, filepos)
      if(filepos > -1) {
        yield line
      }
    }
yield buffer.toString() // yield last line
  } catch(e) {
    console.error("readLine:", e.message)
  } finally {
    fd && fs.closeSync(fd)
  }
}

function readLine(fd: any, buffer: Buffer, bufferSize: number, position: number): [string, number] {
  let line = '', readSize
  const crSize = '\n'.length

  while(true) {
    readSize = fs.readSync(fd, buffer, 0, bufferSize, position)
    if(readSize>0) {
       const temp = buffer.toString('utf8', 0, readSize)
       const index = temp.indexOf('\n')
       if(index>-1) {
        line += temp.substr(0, index)
        position += index + crSize
        break;
       } else {
         line += temp
         position += temp.length
       }
    }
    else {
      position = -1 // end of file
      break
    }
  }
  return [line.trim(), position]
}
```

##### src/1201/fileApi/index.ts

```ts
import {fileExists} from './fileExists'
import {mkdir} from './mkdir'
import {rmdir} from './rmdir'
import {writeFile} from './writeFile'
import {readFile} from './readFile'
import {appendFile} from './appendFile'
import {deleteFile} from './deleteFile'
import {readFileGenerator} from './readFileGenerator'

export {fileExists, mkdir, rmdir, writeFile, readFile, appendFile, deleteFile, readFileGenerator}
```

##### src/1201/test/readFileGenerator-test.ts

```ts
import {readFileGenerator} from '../fileApi'

for(let value of readFileGenerator("data/fake-100000.csv")) {
  console.log('<line>', value, '</line >') // <line> name,email,ip,phone </line >
  break // 시간상 첫 줄만 프린트
}

```

##### src/1201/csv/csvFileReaderGenerator

```ts
import { readFileGenerator } from '../fileApi'
import { zip } from '../utils'

export function* csvFileReaderGenerator(filename: string, delim:string=',') {
  let header = []
  for(let line of readFileGenerator(filename)) {
    if(!header.length) 
      header = line.split(delim)
    else 
      yield zip(header, line.split(delim))
  }
}
```

##### src/1201/readCsv.ts

```ts
import {getFileNameAndNumber} from './utils'
import {csvFileReaderGenerator} from './csv/csvFileReaderGenerator'

const [filename] = getFileNameAndNumber('./data/fake-100000.csv', 1)

let line = 1
for(let object of csvFileReaderGenerator(filename)) {
  console.log(`[${line++}] ${JSON.stringify(object)}`)
}
console.log('\n read complete.')
```

### 12.2 몽고DB에 데이터 저장하기

#### 몽고DB 설치

```bash
scoop install mongodb
mongod --install
net start mongod
```

#### 실습프로젝트 구성
* 1201/fileApi 폴더를 1202/fileApi로 복사

```bash
npm i -S mongodb
npm i -D @types/mongodb
mkdir -p src/1202/mongodb
```
#### 몽고DB 접속하기

##### src/1202/mongodb/connect.ts

```ts
import { MongoClient } from 'mongodb';

export const connect = async (mongoUrl: string = 'mongodb://localhost:27017') => {
  try {
    const client = await MongoClient.connect(mongoUrl);
    console.log('Connected to MongoDB');
    return client;
  } catch (error) {
    console.error('Failed to connect to MongoDB:', error);
    throw error;
  }
};
```

##### src/1202/test/connect-test.ts

```ts
import {connect} from '../mongodb/connect'

const connectTest = async() => {
  let connection
  try {
    connection = await connect()
    console.log('connection OK.', connection)
  } catch(e) {
    console.log(e.message)
  } finally {
    connection.close()
  }  
}

connectTest()
```

#### 데이터베이스 연결
* 몽고DB Location
  - navicat : mongodb > advanced > setting locatioj  = C:\Users\jinny\scoop\persist\mongodb\data

##### src/1202/test/makedb-test.ts

```ts
import {connect} from '../mongodb/connect'

const makedbTest = async() => {
  let connection
  try {
    connection = await connect()
    const db = await connection.db("ch12-2")
    console.log('db', db)
  } catch(e) {
    console.log(e.message)
  } finally {
    connection.close()
  }  
}

makedbTest()
```

##### src/1202/test/collection-test.ts

```ts
import {connect} from '../mongodb/connect'

const makeCollectionsTest = async() => {
  let connection
  try {
    connection = await connect()
    const db = await connection.db("ch12-2")
    const personsCollection  = db.collection("persons")
    const addressesCollection = db.collection("addresses")
    console.log(personsCollection, addressesCollection)
  } catch(e) {
    console.log(e.message)
  } finally {
    connection.close()
  }  
}

makeCollectionsTest()
```

##### src/1202/test/insert-document-test.ts

```ts
import {connect} from '../mongodb/connect'

const insertDocumentTest = async() => {
  let connection, cursor
  try {
    connection = await connect()
    const db = await connection.db("ch12-2")
    const personsCollection  = db.collection("persons")
    const addressesCollection = db.collection("addresses")
    
    const person = {name:"Jack", age: 32}
    let result = await personsCollection.insertOne(person)
    console.log(result)

    const address = {country: "korea", city: "seoul"}
    result = await addressesCollection.insertOne(address)
    console.log(result)
  } catch(e) {
    console.log(e.message)
  } finally {
    connection.close()
  }  
}

insertDocumentTest()
```

#### _id속성과 ObjectID 타입
* 모든 몽고DB문서는 `_id라는 속성, 이 속성은 문서생서시 자동생성`
* 이 값은 ObjectiD('문자열')형태로 출력, 프로그램에서는 `import {ObjectID} from 'mongodb'`로 조회가능

#### 문서찾기
* 특정 컬렉션의 문서들은 다음과 같은 코드로 조회가능
* find메서드는 cursor라는 객체를 리턴하고 cursor.toArray()로 배열을 얻을 수 있다.
```ts
let cursor = await 컬렉션객체.find(검색조건)
const result = await cursor.toArray()
```

##### src/1202/test/find-test.ts

```ts
import {connect} from '../mongodb/connect'

const findDocumentTest = async() => {
  let connection, cursor
  try {
    connection = await connect()
    const db = await connection.db("ch12-2")
    const personsCollection  = db.collection("persons")
    const addressesCollection = db.collection("addresses")
    
    cursor = personsCollection.find({ name: "Jack" })
    const foundPersons = await cursor.toArray()
    console.log(foundPersons) // [ { _id: 5de606f79db96c1ea4c9c81c, name: 'Jack', age: 32 } ]

    cursor = addressesCollection.find({})
    const foundAddresses = await cursor.toArray()
    console.log(foundAddresses) // [ { _id: 5de606f79db96c1ea4c9c81d, country: 'korea', city: 'seoul' } ]
  } catch(e) {
    console.log(e.message)
  } finally {
    connection.close()
  }  
}

findDocumentTest()
```

##### src/1202/test/findOne-test.ts
* findOne() 메서드

```ts
import {connect} from '../mongodb/connect'
import {ObjectId} from 'mongodb'

const findOneTest = async() => {
  let connection, cursor
  try {
    connection = await connect()
    const db = await connection.db("ch12-2")
    const personsCollection  = db.collection("persons")
    
    cursor = personsCollection.find({})
    const foundPersons = await cursor.toArray()
 
    const _id = foundPersons[0]._id
    const result = await personsCollection.findOne({_id})
    console.log(result) // { _id: 5de606f79db96c1ea4c9c81c, name: 'Jack', age: 32 }
  } catch(e) {
    console.log(e.message)
  } finally {
    connection.close()
  }  
}

findOneTest()
```

#### 문서 삭제하기
* deleteOne(), deleteMany()


##### src/1202/test/delete-test.ts
* findOne() 메서드

```ts
import {connect} from '../mongodb/connect'

const deleteTest = async() => {
  let connection
  try {
    connection = await connect()
    const db = await connection.db("ch12-2")
    const personsCollection  = db.collection("persons")
    await personsCollection.insertMany([
      {name: "Jack"}, {name: "Tom"}, {name: "Jane"}
    ])
    
    let result = await personsCollection.deleteOne({name: "Tom"})
    console.log(result) // deleteCount: 1
    result = await personsCollection.deleteMany({})
    console.log(result) // deletedCount: 2
  } catch(e) {
    console.log(e.message)
  } finally {
    connection.close()
  }  
}

deleteTest()
```

#### 문서 정렬하기
* find().sord()


##### src/1202/test/sort-test.ts
* findOne() 메서드

```ts
import {connect} from '../mongodb/connect'

const sortTest = async() => {
  let connection
  try {
    connection = await connect()
    const db = await connection.db("ch12-2")
    const personsCollection  = db.collection("persons")
    await personsCollection.createIndex({name: 1, age: -1})
    await personsCollection.deleteMany({})
    await personsCollection.insertMany([
      {name: "Jack", age: 32}, {name: "Jack", age: 33}, {name:"Jane", age: 10}
    ])

    const cursor = personsCollection.find({ name: "Jack" }).sort({age: -1})
    const result = await cursor.toArray()
    console.log(result) /* [
      { _id: 5de61e788086e31140ac244d, name: 'Jack', age: 33 },
      { _id: 5de61e788086e31140ac244c, name: 'Jack', age: 32 }
    ] */
  } catch(e) {
    console.log(e.message)
  } finally {
    connection.close()
  }  
}

sortTest()
```

#### CSV파일 몽고DB에 저장하기

##### src/1202/insert-csv-to-mongo.ts

```ts
import {connect} from './mongodb/connect'
import {getFileNameAndNumber} from './utils'
import {csvFileReaderGenerator} from './csv/csvFileReaderGenerator'

const insertCsvToMongo = async(csvFilename, collectionName, index) => {
  let connection
  try {
    connection = await connect()
    const db = await connection.db("ch12-2")
    const collection  = db.collection(collectionName)
    await collection.deleteMany({})
    await collection.createIndex(index)
    let line = 1
    for(let object of csvFileReaderGenerator(filename)) {
      await collection.insertOne(object)
      console.log(`${line++} inserted.`)
    }
    console.log('\n insertion complete.')
  } catch(e) {
    console.log(e.message)
  } finally {
    connection.close()
  }  
}

const [filename] = getFileNameAndNumber('./data/fake-100000.csv', 1)
insertCsvToMongo(filename, "users", {birthday: -1, name: 1})
```

#### limit와 skip 메서드
* find().sort().skip().limit(5)

##### src/1202/find-limit-skip.ts

```ts
import { connect } from './mongodb/connect'
import { IFake } from './fake'

const findLimitSkip = async () => {
  let connection, cursor
  try {
    connection = await connect()
    const db = await connection.db('ch12-2')
    const usersCollection = db.collection('users')

    let cursor = await usersCollection.find({}).sort({ birthday: -1, name: 1 }).skip(100).limit(5)
    let result = await cursor.toArray()
    console.log(result.map((user: IFake) => ({ name: user.name, birthday: user.birthday })))
  } catch (e) {
    console.log(e.message)
  } finally {
    connection.close()
  }
}

findLimitSkip()
```

### 1203. express로 api서버 만들기

#### 실습프로젝트 구성
* 1202 폴더를 1203폴더로 복사

```bash
npm i -S express body-parser cors
npm i -D @types/express @types/bodt-parser @types/cors
```

#### 익스프레스 프레이워크
* nodejs환경에서 API서버는 대부분 express framework를 사용

##### src/1203/test/express-test.ts
* http://localhost:4000/

```ts
import express from 'express'
const app = express(), port = 4000

app
  .get('/', (req, res) => res.json({message: "Hello Express??"}))
  .listen(port, () => console.log(`http://localhost:${port} started...`))
```

#### 라우팅기능 구현

##### src/1203/test/express-routing-test.ts
* http://localhost:4000/users/1/2

```ts
import express from 'express'
const app = express(), port = 4000

app
  .get('/', (req, res) => res.json({message: "Hello Express??"}))
  .listen(port, () => console.log(`http://localhost:${port} started...`))
```

#### express middleware 추가
* REST API서버들른 웹페이지 본문내용을 분석하려고 할 때 bodyParser와 cors패키지를 use메서드를 사용

```ts
import bodyParser from 'body-parser'
import cors form 'cors'

app
  .use(bodyParser.urlencoded({extended: true}))
  .user(cors())
```

#### MongoDB 연결

##### src/1203/runServer.ts

```ts
import express from 'express'
import bodyParser from 'body-parser'
import cors from 'cors'

export const runServer = (mongodb) => {
  const app = express(), port = 4000
  
  app
  .use(bodyParser.urlencoded({extended: true}))
  .use(cors())
  .get('/', (req, res) => res.json({message: 'Hello world!'}))
  .get('/users/:skip/:limit', async (req, res) => {
    const {skip, limit} = req.params

    const usersCollection = await mongodb.collection("users")
    const cursor = await usersCollection
      .find({})
      .sort({name: 1})
      .skip(parseInt(skip))
      .limit(parseInt(limit))
    const result = await cursor.toArray()
    res.json(result)
  })
  .listen(port, ()=> console.log(`http://localhost:${port} started...`))
}
```

##### src/1203/index.ts
* ts-node src/1203/index.ts
* http://localhost:4000/users/1/2

```ts
import {connect} from './mongodb/connect'
import {runServer} from './runServer'

connect()
  .then( async(connection) => {
    const db = await connection.db("ch12-2")
    return db
  })
  .then(runServer)
  .catch((e: Error) => console.log(e.message))
```



















### 1204. react, bootstrap으로 frontApp 만들기

#### 실습프로젝트 구성

```bash
ts-node src/1203/index.ts

npx create-react-app 13.react-app-users --template typesript
cd 13.react-app-users
npm start
```

#### API서버에서 실제 데이터 가져오기

##### src/IUser.ts

```ts
export interface IUser {
  _id: string
  name: string,
  email: string, 
  sentence: string,
  profession: string,
  birthday: Date,
}
```

##### src/getDataPromise.ts

```ts
import {IUser} from './IUser'

type GetDataPromiseCallback = (a: IUser[]) => void;
export const getDataPromise = (fn: GetDataPromiseCallback) => (skip:number, limit: number) => 
  fetch(`http://localhost:4000/users/${skip}/${limit}`)
  .then(res => res.json())
  .then(fn)

```
##### public/index.html

```html
<!DOCTYPE html>
<html lang="en">
  <head>
    <meta charset="utf-8" />
    <link rel="icon" href="%PUBLIC_URL%/favicon.ico" />
    <meta name="viewport" content="width=device-width, initial-scale=1" />
    <meta name="theme-color" content="#000000" />
    <meta
      name="description"
      content="Web site created using create-react-app"
    />
    <link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/bootstrap/4.3.1/css/bootstrap.min.css" integrity="sha384-ggOyR0iXCbMQv3Xipma34MD+dH/1fQ784/j6cY/iJTQUOhcWr7x9JvoRxT2MZw1T" crossorigin="anonymous">
    <link rel="apple-touch-icon" href="logo192.png" />
    <link rel="manifest" href="%PUBLIC_URL%/manifest.json" />
    <title>React App</title>
  </head>
  <body>
    <noscript>You need to enable JavaScript to run this app.</noscript>
    <div id="root"></div>
   
    <script src="https://stackpath.bootstrapcdn.com/bootstrap/4.4.1/js/bootstrap.bundle.min.js" integrity="sha384-6khuMg9gaYr5AxOqhkVIODVIvm9ynTT5J4V1cfthmT+emCG6yVmEZsRHdxlotUnm" crossorigin="anonymous"></script>
  </body>
</html>
```

##### src/Card.tsx

```ts
import React from "react";
import { IUser } from "./IUser";

const random = (max: number) => Math.floor(Math.random() * max);

export const Card: React.FC<{ user: IUser; click: () => void }> = ({
  user,
  click,
}) => {
  const { name, email, sentence, profession, birthday } = user;
  const b = new Date(birthday);
  const src = `https://source.unsplash.com/random/1000x${random(300) + 500}`;

  return (
    <div className="card">
      <img src={src} className="card-img-top" alt="" />
      <div className="card-body">
        <h5 className="card-title">
          {name} ({email})
        </h5>
        <h6 className="card-subtitle mb-2 text-muted">
          {profession} birthday: {b.getFullYear()}
        </h6>
        <p className="card-text">{sentence}</p>
        <button className="btn btn-primary" onClick={click}>
          more data...
        </button>
      </div>
    </div>
  );
};
```


##### src/App.css

```css
.App {
  width: 100%;
  background-color: white;
  padding: 5px;
  display: flex;
  flex-wrap: wrap;
}

.card {
  width: 22rem;
  padding: 5px;
  margin: 5px;
}
```

##### src/App.tsx

```ts
import React, { useState, useEffect } from "react";
import { IUser } from "./IUser";
import { getDataPromise } from "./getDataPromise";
import { Card } from './Card'
import "./App.css";

const App: React.FC = () => {
  const limit = 2;
  const [skip, setSkip] = useState(0);
  const [users, setUsers] = useState<IUser[]>([]);
  const onClick = () => {
    getDataPromise((receivedUsers: IUser[]) => {
      //setSkip(skip + limit);
      setSkip((skip) => skip + limit);
      //setUsers([...users, ...receivedUsers]);
      setUsers((users) => [...users, ...receivedUsers]);
    })(skip, limit);
  };
  // eslint-disable-next-line
  useEffect(onClick, []);

  return (
    <div className="App">
      {users.map((user: IUser, key: number) => (
        <Card click={onClick} user={user} key={key.toString()} />
      ))}
    </div>
  );
};

export default App;
```